<a href="https://colab.research.google.com/github/joaochenriques/IST---Marine-Currents-and-Tidal-Energy/blob/main/Tidal_Range/SimulEbbGeneration_2025_V07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as mpl
import numpy as np
from numpy.typing import NDArray
from typing import Callable, List, Tuple, Dict
import ipywidgets
import pathlib, os, sys

try:
    from dataclassy import dataclass
except ModuleNotFoundError:
    os.system( "pip install dataclassy" )
    from dataclassy import dataclass

If you are running Python on a Windows operating system, you may need to copy the file

https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py

to the working folder before running the notebook.

To generate a latex file use (win/linux/mac):

```jupyter nbconvert --to latex SimulEbbGeneration_2025_V06.ipynb```

in the command line.

In [ ]:
if not pathlib.Path("mpl_utils.py").exists():
    os.system( "curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py" )

import mpl_utils as mut
from  mpl_utils import linecolors
mut.config_plots()

mpl.rcParams["figure.figsize"] = (12, 3)

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

# Configuration class

All config data is stored in this class

In [ ]:
@dataclass
class Config:

    #===========================================================================
    # turbine data
    no_turb: int = 23           # [-] number of turbines
    D_turb: float = 5           # [m] turbine diameter
    CD_turb: float = 0.65       # [-] turbine discharge coeff as sluice gate
    N_turb: float = np.nan      # [rad/s] synchronous rotational speed

    #===========================================================================
    # generator data
    grid_freq: float = 50.0     # [Hz] electrical grid frequency
    no_ppoles: int = 32         # [-]  Number of pole pairs of the generator
    Pgen_rated: float = 20E6    # [W]  generator rated power

    #===========================================================================
    # sluice gates data
    no_sgates: int = 10         # [-]  number of sluice gates
    A_sgates: float = 10.0*15.0 # [m²] sluice gates area
    CD_sgates: float = 0.65     # [-]  sluice gates discharge coeff

    #===========================================================================
    # coeffs of the quadratic polynomial that describes the basin area
    # [ a, b, c ] => a*z**2 + b*z + c
    # The zero corresponds to the mean water level
    basin_coeffs = np.array( [ -0.102996E6, 1.272972E6, 23.31E6 ] )
    basin_z0 = 1.0              # [m] initial basin level

    #===========================================================================
    # tide components
    A: NDArray = np.array( [ 4.18, 1.13 ] )               # [m] amplitudes
    ω: NDArray = np.array( [ 0.5058/3600, 0.5236/3600 ] ) # [rad/s] frequencies
    φ: NDArray = np.array( [ -3.019, -3.84 ] )            # [rad] phases
    global_period: float        # [s] simulations' repetition period

    #===========================================================================
    delta_t = 100.0             # [s] integration time step
    simul_time: float = np.nan  # [s]

    #===========================================================================
    # state-space config - number of variables and outputs
    # DO NOT CHANGE!
    n_SS_vars = 4
    n_SS_outs = 11

    #===========================================================================
    # Minimum turbine head that starts turbine generation.
    # Can be time dependent instead of a constant.
    def turbine_starting_head( self, t: float ) -> float:
        return 5.8

    #===========================================================================
    def __init__( self ):

        # change if more than two tidal waves are used in the simulations
        self.N_turb = 2 * np.pi * self.grid_freq / self.no_ppoles

        self.global_period = 2.0 * np.pi / ( np.max( self.ω ) - np.min( self.ω ) )
        self.simul_time = 4.0 * self.global_period

        if len( self.ω ) == 2:
            print( 'WARNING: global period is not exact for more than 2 frequencies' )

## Tide modelling

The tide level is assumed to be a trignometric series

$$\zeta(t) = \sum_i^n A_i \cos\left( \omega_i t + \phi_i \right).$$

In [ ]:
@dataclass(slots=True)
class TideModel:

    A_tide: NDArray
    ω_tide: NDArray
    φ_tide: NDArray

    # A_tide, period and φ_tide are vectors to allow simulate multi-component tides
    def __init__( self, A_tide: NDArray, ω_tide: NDArray, φ_tide: NDArray ) -> None:
        self.A_tide = A_tide
        self.ω_tide = ω_tide
        self.φ_tide = φ_tide


    # tide level as a function t
    def level( self, t: float ) -> float:
        return float( np.sum( self.A_tide * np.cos( self.ω_tide * t + self.φ_tide ) ) )

# Plot tidal time series with envelope in case of two frequencies

In [ ]:
cfg = Config()

if len( cfg.ω ) == 2:
    sec2day = 24 * 3600.0

    tide = TideModel( cfg.A, cfg.ω, cfg.φ )

    period = cfg.global_period
    ev_time_vec = np.linspace( 0, 1.2 * period, int( period / cfg.delta_t ) )
    ev_ζ_vec = np.zeros_like( ev_time_vec )

    for i, t in enumerate(ev_time_vec):
        ev_ζ_vec[i] = tide.level( t )

    X1 = cfg.A[0]
    X2 = cfg.A[1]
    ωm = cfg.ω[0] - cfg.ω[1]
    φm = cfg.φ[0] - cfg.φ[1]
    ev = np.sqrt( X1**2 + X2**2 + 2 * X1 * X2 * np.cos( ωm * ev_time_vec + φm ) )

    ev_time_vec /= sec2day

    dx = np.ptp( ev_time_vec )
    xmin = ev_time_vec[ 0] - dx * 0.01
    xmax = ev_time_vec[-1] + dx * 0.01

    def update( change: Tuple[float, float] ) -> None:

        fig, ax = mpl.subplots()
        ax.plot( ev_time_vec, ev_ζ_vec, label = "tide level" )
        ax.plot( ev_time_vec, ev, 'r-', lw = 2, label = "envelop" )
        ax.legend( loc = 'lower left' )
        ax.set_xlabel( 'time [day]' )
        ax.set_xlim( *change )

        fig.savefig( 'TimeSeriesTides.pdf', bbox_inches = 'tight', pad_inches = 0 ) # pdflatex
        fig.savefig( 'TimeSeriesTides.svg', bbox_inches = 'tight', pad_inches = 0 ) # Word

    ipywidgets.interact( update,
                         change = ipywidgets.FloatRangeSlider(
                                                value=(xmin,xmax),
                                                min=xmin, max=xmax, step=0.1,
                                                description = "Interval"
                                            )
                        )

    print( f'Period: {period / sec2day:.1f}day, Neap tide amp: {np.min(ev):.2f}m, Spring tide amp: {np.max(ev):.2f}m' )
else:
    print( "WARNING: Envelop only available for two tidal waves." )

# General description of the power plant's models

## Barrage simulator in ebb mode

The following models were implemented to simulate the tidal power plant:

* The basin
* The tide
* Hydraulic turbines
* Electrical generators
* Sluice gates
* Power plant controller

## Turbine model
### Turbine hill map and turbine operating curve

The turbine model uses the maximum power curve, ie., the red and gree lines of the turbine hill map.

The isolines of the efficiency are plotted as $\eta/\eta_\text{bep}$, where $\eta_\text{bep}=0.912$.

<img src="https://raw.githubusercontent.com/joaochenriques/IST_MCTE/main/Barrages/TurbineGeneratorMaps/TurbineHill_Plot.svg" width="500px" style="display:inline">


### Turbine mode

Turbine dimensionless numbers

* Rotational speed $$n_{11}=\dfrac{ND}{\sqrt{gh}}=\dfrac{C_0}{\sqrt{h}}.$$

* Flow rate $$Q_{11} = \dfrac{Q}{D^2\sqrt{gh}}=\dfrac{Q}{C_1\sqrt{h}}.$$

* Efficiency $$\eta_\mathrm{turb} = \dfrac{P_\mathrm{turb}}{P_\mathrm{avail}}.$$

where $C_0=N D / \sqrt{g}$ and $C_1=D^2\sqrt{g}$.


The power available to the turbine is given by

$$P_\mathrm{avail} = \rho g h Q = \rho N^3 D^5 \dfrac{Q_{11}}{n_{11}^3},$$

where

$$Q = D^2 \sqrt{gh} \, Q_{11}\!\big( n_{11} \big)=C_1 \sqrt{h} \, Q_{11}\!\big( n_{11} \big).$$

The turbine is to be operated at constant rotational speed due to the use of a synchronous generator (see generator class).

The available energy is

$$\dfrac{\text{d} E_\text{avail}}{\text{d}t} = P_\text{avail}.$$

The energy harvest by the turbine is

$$\dfrac{\text{d} E_\text{turb}}{\text{d}t}=\eta_\text{turb}\big( n_{11}\big) \, P_\text{avail},$$

The mean turbine efficiency is

$$\overline{\eta_\text{turb}} = \dfrac{E_\text{turb}}{E_\text{avail}}.$$

### Sluicing mode

The in sluicing mode the "turbine" is modelled as

$$Q_\mathrm{turb}^\mathrm{sluice} = C_\mathrm{d} A_\mathrm{turb} \sqrt{ 2 g h }$$

where $A_\mathrm{turb}$ is the area corresponding to the turbine rotor diameter.

In [ ]:
@dataclass
class TurbineModel:

    # flow rate: red line of the map
    poly_CQ1 = np.poly1d( np.array( [ 0.16928201, 0.08989368 ] ) )

    # flow rate: green line of the map
    poly_CQ2 = np.poly1d( np.array( [ -3.63920467e-04,  9.37677378e-03,
                                      -9.25873626e-02,  1.75687197e+00 ] ) )

    # efficiency: red line of the map
    poly_CE1 = np.poly1d( np.array( [ -0.02076456, 0.20238444, 0.48984553 ] ) )
    # efficiency: green line of the map
    poly_CE2 = np.poly1d( np.array( [ -2.75685709e-04,  2.04822984e-03,
                                       6.86081825e-04,  7.93083108e-01 ] ) )

    η_max: float = 0.912  # [-] maximum efficiency

    # n11 interpolation domain
    n11_min: float =  4.38    # [-]
    n11_max: float = 17.17    # [-]
    n11_r2g: float =  7.92193 # [-] red to green point

    # other data
    gr: float = 9.8         # [m/s²] gravity aceleration
    ρw: float = 1025.0      # kg/m³] water density
    CD_turb: float          # [-] sluice mode discharge coefficient
    C_0: float = np.nan     # constant used to compute dimensionless velocity
    C_1: float = np.nan     # constant used to compute dimensionless flow rate


    #=============================================================================
    def __init__( self, D_turb, N_turb, CD_turb ) -> None:

        self.N_turb = N_turb  # we are assuming constant rotational speed model
        self.D_turb = D_turb  # turbine rotor diameter
        self.A_turb = np.pi * ( D_turb / 2.0 )**2
        self.CD_turb = CD_turb

        # constants used in for computing n11 and QT
        self.C_0 = N_turb * D_turb / np.sqrt( self.gr )
        self.C_1 = D_turb**2 * np.sqrt( self.gr )


    # turbine operating range
    def n11_range( self ) -> tuple:
        return ( self.n11_min, self.n11_max )


    # dimensionless velocity
    def n11( self, h: float ) -> float:
        # avoid division by zero on h = 0.0
        return self.C_0 / np.sqrt( np.maximum( h, 1E-3 ) )


    # dimensionless flow rate
    def Q11( self, n11: float ) -> float:
        assert( n11 >= self.n11_min ), "n11 small than admissable minimum"
        assert( n11 <= self.n11_max ), "n11 greater than admissable maximum"
        if n11 < self.n11_r2g:
            return self.poly_CQ1( n11 )
        else:
            return self.poly_CQ2( n11 )


    # efficiency
    def eta( self, n11: float ) -> float:
        assert( n11 >= self.n11_min ), "n11 small than admissable minimum"
        assert( n11 <= self.n11_max ), "n11 greater than admissable maximum"
        if n11 < self.n11_r2g:
            return self.poly_CE1( n11 ) * self.η_max
        else:
            return self.poly_CE2( n11 ) * self.η_max


    # computing operational data
    def operating_point( self, h: float ) -> Tuple:
        n11 = self.n11( h )
        Q11 = self.Q11( n11 )
        Q_turb  = self.C_1 * Q11 * np.sqrt( h )
        P_avail = self.ρw * self.gr * h * Q_turb
        η_turb  = self.eta( n11 )
        return ( Q_turb, P_avail, η_turb, n11, Q11 )


    # turbine flow rate in sluice mode
    def sluicing( self, h: float ) -> float:
        QS = -self.CD_turb * self.A_turb * \
                    np.sqrt( 2.0 * self.gr * np.maximum( -h, 0.0 ) )
        return QS

## Generator efficiency curve

The electrical generator is assumed to be a synchronous machine. The rotational speed is given by

$$\Omega = \dfrac{2\pi f_\mathrm{g}}{n_\mathrm{p}}$$

where $f_\mathrm{g}$ is the electrical grid frequency and $n_\mathrm{p}$ is the number of pairs of poles.

<br />

The generator efficiency is computed as a function of the load

$$\Lambda = \dfrac{P_\mathrm{turb}}{P_\mathrm{gen}^\mathrm{rated}}$$


<img src="https://raw.githubusercontent.com/joaochenriques/MCTE_2022/main/Barrages/TurbineGeneratorMaps/GeneratorEff_plot.svg" width="400px" style="display:inline">

Electrical power output is

$$P_\mathrm{gen} =\eta_\mathrm{gen}(\Lambda) \, P_\mathrm{turb},$$

and the converted energy is

$$\dfrac{\mathrm{d} E_\mathrm{gen}}{\mathrm{d} t} = P_\mathrm{gen}.$$

The mean generator efficiency is

$$\overline{\eta_\mathrm{gen}}=\dfrac{E_\mathrm{gen}}{E_\mathrm{turb}}.$$

In [ ]:
@dataclass(slots=True)
class GeneratorModel:

    # red part of the curve
    poly_C1: np.poly1d
    # green part of the curve
    poly_C2: np.poly1d
    # generator rated power
    Pgen_rated: float


    def __init__( self, Pgen_rated: float ) -> None:
        # red part of the curve
        self.poly_C1 = np.poly1d( np.array( [ -6.71448631e+03,  2.59159775e+03,
                                              -3.80834059e+02,  2.70423225e+01,
                                               3.29394948e-03 ] ) )
        # green part of the curve
        self.poly_C2 = np.poly1d( np.array( [ -1.16856952, 3.31172525,
                                              -3.44296217, 1.54160290,
                                               0.71040716 ] ) )
        self.Pgen_rated = Pgen_rated


    # efficiency as a function of the load
    def eta( self, P_turb: float ) -> float:
        load = P_turb / self.Pgen_rated

        assert( load >= 0.0 ),  "turbine power lower than zero"
        assert( load <= 1.0 ), f"generator rated power too low ({P_turb})"
        if load < 0.12542:
            return self.poly_C1( load )
        else:
            return self.poly_C2( load )

## Sluice gates

The sluice gates are modelled as a turbulent pressure drop

$$Q_\mathrm{sluice} = C_\mathrm{d} A \sqrt{ 2 g h }$$

typical discharge coefficients for barrage sluice gates are within the range $0.6 \le C_\mathrm{d} \le 1.2$.

In the follwoing model, it is assumed that the sluice gates are closed when the sea level is below the basin level $(h\ge 0)$.

In [ ]:
@dataclass(slots=True)
class SluiceGateModel:

    gr: float          # [m/s²] gravity acceleration
    A_sgates: float    # [m²] Area
    CD_sgates: float   # [-] discharge coefficient


    def __init__( self, A_sgates: float, CD_sgates ) -> None:
        self.gr = 9.8
        self.A_sgates = A_sgates    # Area
        self.CD_sgates = CD_sgates  # discharge coefficient


    # flow rate as a function of h
    def sluicing( self, h: float ) -> float:
        QS = -self.CD_sgates * self.A_sgates * np.sqrt( 2.0 * self.gr * max( -h, 0.0 ) )
        return QS

## Basin modelling


<img src="https://raw.githubusercontent.com/joaochenriques/MCTE_2022/main/Barrages/Figures/BasinShoeBox_MWL.svg" width="300px" style="display:inline">


<br/>


The instantaneous basin volume is computed from

$$\dfrac{\mathrm{d}V}{\mathrm{d}t}=-Q(t).$$

The out flow is denoted as positive.

<br />

Let $z(t)$ be the water level with respect to the mean water level (MWL) and $A_\mathrm{basin}(z)$ the basin area as a function of $z(t)$.

<br />

Knowing that $V=z \, A_\mathrm{basin}(z)$, the basin volume change as function of time is computed as

$$
\dfrac{\mathrm{d}V}{\mathrm{d}t}
=\dfrac{\mathrm{d}(z \, A_\mathrm{basin}(z))}{\mathrm{d}t}
=\dfrac{\mathrm{d}z}{\mathrm{d}t}A_\mathrm{basin}(z) + z\,\dfrac{A_\mathrm{basin}(z)}{\mathrm{d}z}\dfrac{\mathrm{d}z}{\mathrm{d}t}
= f_\mathrm{basin}(z)\dfrac{\mathrm{d}z}{\mathrm{d}t}
$$

where

$$f_\mathrm{basin}(z)=A_\mathrm{basin}(z) + z\,\dfrac{A_\mathrm{basin}(z)}{\mathrm{d}z},$$

giving

$$\dfrac{\mathrm{d}z}{\mathrm{d}t}=-\frac{Q(t)}{f_\mathrm{basin}(z)}.$$

<br/>

Integrating with the Euler method in $\Delta t$, we get

$$z(t+\Delta t)=z(t) + \Delta t\,\left(-\frac{Q(t)}{f_\mathrm{basin}(z(t))}\right).$$


## Power plant operation model

The power plant modelling and control is based in the following finite state machine

<img src="https://raw.githubusercontent.com/joaochenriques/MCTE_2022/main/Barrages/Figures/BarrageEbbMode.svg" width="700px" style="display:inline">

<img src="https://raw.githubusercontent.com/joaochenriques/MCTE_2022/main/Barrages/Figures/EbbOperation_FSM.svg" width="700px" style="display:inline">

The numerical model of the the tidal range power plant is defined by the set of ODEs:

$$\dfrac{ \mathrm{d} \mathbf{x} }{\mathrm{d} t}=\mathbf{f}(t,\mathbf{x}),$$

where the variables are:

$$ \mathbf{x} =
\begin{pmatrix} z & E_\mathrm{avail} & E_\mathrm{turb} & E_\mathrm{gen} \end{pmatrix}^\mathrm{T} $$

and the RHS is given by

$$
\mathbf{f}(t,\mathbf{x})
=
\begin{pmatrix}
\displaystyle -\frac{Q(t)}{f_\mathrm{basin}(z)}\\[2pt]
P_\mathrm{avail}\\
P_\mathrm{turb}\\
P_\mathrm{gen}\\
\end{pmatrix}
$$


The outputs are:

$$ \mathbf{y} =
\begin{pmatrix} h & \zeta & Q_\mathrm{turb} & Q_\mathrm{sluice} &
P_\mathrm{avail} & P_\mathrm{turb} & P_\mathrm{gen} &
\eta_\mathrm{turb} & \eta_\mathrm{gen} \end{pmatrix}^\mathrm{T} $$

# Finite State Machine simulator

In [ ]:
@dataclass(slots=True)
class FSM_simulator:

    # Callable[ list of function args, function output ]
    func_state = Callable[ [ float, float, NDArray ], Tuple ]
    func_transition = Callable[ [ float, NDArray, NDArray ], str ]

    dic_states: Dict
    dic_transitions: Dict

    n_time: int
    n_SS_vars: int
    n_SS_outs: int

    time: NDArray
    delta_t: float

    SS_vars: NDArray
    SS_outs: NDArray

    states_ID: NDArray
    dic_Plot_ID: Dict


    def __init__( self, simul_time: float, Delta_t: float, n_SS_vars: int, n_SS_outs: int ) -> None:
        '''
        Allocates all data storage
        '''
        self.dic_states = {}
        self.dic_transitions = {}

        self.n_time = int( simul_time / Delta_t + 0.5 )
        self.n_SS_vars = n_SS_vars
        self.n_SS_outs = n_SS_outs

        self.time = np.linspace( 0, simul_time, self.n_time )
        self.delta_t = self.time[1]

        self.SS_vars = np.zeros( (n_SS_vars, self.n_time) )
        self.SS_outs = np.zeros( (n_SS_outs, self.n_time) )

        self.states_ID = np.zeros( self.n_time )
        self.dic_Plot_ID = {}


    def run_simulator( self, state: str, SS_vars: NDArray ) -> Tuple:
        '''
        Simulates the process for the specified time interval
        '''
        self.SS_vars[:,0] = SS_vars
        self.states_ID[0] = self.dic_Plot_ID[state]
        # outs not available at t=0

        for i in range( 1, self.n_time ):

            SS_vars, SS_outs = self.dic_states[ state ]( self.delta_t, self.time[i-1], SS_vars )
            state = self.__test_transitions( state, self.time[i-1], SS_vars, SS_outs )

            self.SS_vars[:,i] = SS_vars
            self.SS_outs[:,i] = SS_outs
            self.states_ID[i] = self.dic_Plot_ID[state]

        return ( self.time, self.states_ID, self.SS_vars, self.SS_outs )


    def add_state( self, state: str, Plot_ID: int, func: func_state ) -> None:
        '''
        state (str) is the key to store the Plot_ID and func in the dics
        plot_ID (int) is used to generate the time series with the current state
        func_state
        '''
        assert state not in self.dic_states, f"State '{state}' is already defined."
        self.dic_Plot_ID[state] = Plot_ID
        self.dic_states[state] = func


    def add_transition_to_state( self, state: str, func: func_transition ) -> None:
        '''
        At the end of the state time step, transitions are tested in the order they were added
        '''
        self.dic_transitions.setdefault( state, [] ).append( func )


    # private member
    def __test_transitions( self, state: str, t: float, vars: NDArray, outs: NDArray ) -> str:
        '''
        At the end of each time step, all transitions are tested until one
        successfully changes the current state
        '''
        for transition in self.dic_transitions[ state ]:
            next_state = transition( t, vars, outs )
            if next_state != state:
                return next_state # first transition that changes the state
        return state

# Finite states and transitions used to model the power plant

In [ ]:
@dataclass(slots=True)
class PowerPlantModel:

    #===========================================================================
    # declare a type alias for a function of a z that returns a float
    z_func = Callable[ [ float ], float ]

    #===========================================================================
    N: float        # [rad/s] synchronous rotational speed
    n11_max: float  # [-] used in the transitions
    n_turbs: int    # [-]
    n_gates: int    # [-]

    #===========================================================================
    # component models
    turbine: TurbineModel
    generator: GeneratorModel

    gate: SluiceGateModel
    tide: TideModel

    #===========================================================================
    # basin data and turbine starting head
    basin_Area: np.poly1d
    basin_dArea_dz: np.poly1d
    turbine_starting_head: z_func

    #===========================================================================
    FS_order: Dict # plotting order

    #===========================================================================
    def __init__( self, cfg: Config ) -> None:

        self.n_turbs = int( cfg.no_turb )
        self.n_gates = int( cfg.no_sgates )

        self.turbine = TurbineModel( cfg.D_turb, cfg.N_turb, cfg.CD_turb )
        self.generator = GeneratorModel( Pgen_rated = cfg.Pgen_rated )

        _, self.n11_max = self.turbine.n11_range()

        self.gate = SluiceGateModel( cfg.A_sgates, cfg.CD_sgates )
        self.tide = TideModel( cfg.A, cfg.ω, cfg.φ )

        self.basin_Area = np.poly1d( cfg.basin_coeffs )
        self.basin_dArea_dz = self.basin_Area.deriv(1)

        self.turbine_starting_head = cfg.turbine_starting_head

        self.FS_order = { "(S0)":0, "(S1)":1, "(S2)":2, "(S3)":3, "(S4)":4 }


    #===========================================================================
    def basin_func( self, z: float ) -> float:
        return self.basin_Area(z) + self.basin_dArea_dz(z) * z


    ## STATES ##################################################################
    def S1_Generate( self, delta_t: float, t: float, SS_vars: NDArray ) -> Tuple:

        z = SS_vars[0]
        ζ = self.tide.level( t )
        h = z - ζ

        Q_turb, P_avail, η_turb, _, _ = self.turbine.operating_point( h )
        P_turb = η_turb * P_avail

        η_gen = self.generator.eta( P_turb )
        P_gen  = η_gen * P_turb

        q_basin = Q_turb * self.n_turbs / self.basin_func( z )
        RHS = np.array( ( -q_basin, P_avail, P_turb, P_gen ) )

        # variables at ( t + delta_t )
        # First order Euler scheme  (explicit). Can be improved!
        SS_vars += delta_t * RHS

        #=========================================================================
        # outputs at ( t + delta_t )
        z = SS_vars[0]
        ζ = self.tide.level( t + delta_t )
        h = z - ζ

        Q_turb, P_avail, η_turb, n11, Q11 = self.turbine.operating_point( h )
        P_turb = η_turb * P_avail

        η_gen = self.generator.eta( P_turb )
        P_gen  = η_gen * P_turb

        Q_sluice = 0.0

        SS_outs = np.array( ( h, ζ, Q_turb, Q_sluice, P_avail, P_turb, P_gen, η_turb, η_gen, n11, Q11 ) )

        return ( SS_vars, SS_outs )


    def SX_Hold( self, delta_t: float, t: float, SS_vars: NDArray ) -> Tuple:

        # outputs at ( t + delta_t )
        z = SS_vars[0]
        ζ = self.tide.level( t + delta_t )
        h = z - ζ

        Q_turb = Q_sluice = P_avail = P_turb = P_gen = η_turb = η_gen = 0.0

        SS_outs = np.array( ( h, ζ, Q_turb, Q_sluice, P_avail, P_turb, P_gen, η_turb, η_gen, 0, 0 ) )

        # stats do not change
        return ( SS_vars, SS_outs )


    def S3_Fill( self, delta_t: float, t: float , SS_vars: NDArray ) -> Tuple:

        z = SS_vars[0]
        ζ = self.tide.level( t )
        h = z - ζ

        Q_sluice = self.gate.sluicing( h )
        Q_turb = self.turbine.sluicing( h )

        q_sluice = Q_sluice * self.n_gates
        q_turb = Q_turb * self.n_turbs
        q_basin = ( q_sluice + q_turb ) / self.basin_func( z )
        RHS = np.array( ( -q_basin, 0.0, 0.0, 0.0 ) )

        # variables at ( t + delta_t )
        # First order Euler scheme  (explicit). Can be improved!
        SS_vars += delta_t * RHS

        #=========================================================================
        # outputs at ( t + delta_t )
        z = SS_vars[0]
        ζ = self.tide.level( t + delta_t )
        h = z - ζ

        Q_sluice = self.gate.sluicing( h )
        Q_turb = self.turbine.sluicing( h )

        P_avail = P_turb = P_gen = η_turb = η_gen = 0.0

        SS_outs = np.array( ( h, ζ, Q_turb, Q_sluice, P_avail, P_turb, P_gen, η_turb, η_gen, 0, 0 ) )

        return ( SS_vars, SS_outs )


    ## TRANSITIONS #############################################################
    def T_S0_S1( self, t: float, SS_vars: NDArray, SS_outs: NDArray ) -> str:
        h = SS_outs[0]
        h_start = self.turbine_starting_head( t )
        return "(S1)" if h > h_start else "(S0)"


    def T_S1_S2( self, t: float, SS_vars: NDArray, SS_outs: NDArray ) -> str:
        h = SS_outs[0]
        n11 = self.turbine.n11( h )
        # Use a 10% safety factor to avoid exceeding limits
        return "(S2)" if n11*1.1 > self.n11_max else "(S1)"


    def T_S2_S3( self, t: float, SS_vars: NDArray, SS_outs: NDArray ) -> str:
        h = SS_outs[0]
        return "(S3)" if h < 0.0 else "(S2)"


    def T_S3_S4( self, t: float, SS_vars: NDArray, SS_outs: NDArray ) -> str:
        h = SS_outs[0]
        return "(S4)" if h > 0.0 else "(S3)"


    def T_S4_S1( self, t: float, SS_vars: NDArray, SS_outs: NDArray ) -> str:
        h = SS_outs[0]
        h_start = self.turbine_starting_head( t )
        return "(S1)" if h > h_start else "(S4)"

In [ ]:
PP_model = PowerPlantModel( cfg )

PP_simul = FSM_simulator( cfg.simul_time, cfg.delta_t, cfg.n_SS_vars, cfg.n_SS_outs )

PP_simul.add_state( "(S0)", 0, PP_model.SX_Hold )
PP_simul.add_state( "(S1)", 1, PP_model.S1_Generate )
PP_simul.add_state( "(S2)", 2, PP_model.SX_Hold )
PP_simul.add_state( "(S3)", 3, PP_model.S3_Fill )
PP_simul.add_state( "(S4)", 4, PP_model.SX_Hold )

PP_simul.add_transition_to_state( "(S0)", PP_model.T_S0_S1 )
PP_simul.add_transition_to_state( "(S1)", PP_model.T_S1_S2 )
PP_simul.add_transition_to_state( "(S2)", PP_model.T_S2_S3 )
PP_simul.add_transition_to_state( "(S3)", PP_model.T_S3_S4 )
PP_simul.add_transition_to_state( "(S4)", PP_model.T_S4_S1 )

print( f'Basin area(z=0) = {PP_model.basin_Area(0):.2E}' )

# Simulate the power plant

In [ ]:
initial_condition = np.array( ( cfg.basin_z0, 0.0, 0.0, 0.0 ) )

time_vec, FSM_states_vec, SS_vars, SS_outs = PP_simul.run_simulator( "(S0)", initial_condition )

#=================================
z_vec       = SS_vars[0]
E_avail_vec = SS_vars[1]
E_turb_vec  = SS_vars[2]
E_gen_vec   = SS_vars[3]

#=================================
h_vec = SS_outs[0]
ζ_vec = SS_outs[1]

Q_turb_vec   = SS_outs[2]
Q_sluice_vec = SS_outs[3]

P_avail_vec = SS_outs[4]
P_turb_vec  = SS_outs[5]
P_gen_vec   = SS_outs[6]
η_turb_vec  = SS_outs[7]
η_gen_vec   = SS_outs[8]

n11_vec     = SS_outs[9]
Q11_vec     = SS_outs[10]

In [ ]:
hours_vec = time_vec / 3600.0
period_hours = cfg.global_period / 3600.0

dx = np.ptp( hours_vec ) # peak to peak
xmin = np.round( hours_vec[ 0] - dx * 0.01 )
xmax = np.round( hours_vec[-1] + dx * 0.01 )
xdelta = ( xmin + dx * 0.43, xmax - dx * 0.43 )

# Number of points of each period. Required to make the mean of last period
pp = int( cfg.global_period / cfg.delta_t )

P_turb_max = np.max( P_turb_vec )
P_turb_mean = np.mean( P_turb_vec[-pp:] )
P_gen_mean = np.mean( P_gen_vec[-pp:] )
C_fac = P_gen_mean / PP_model.generator.Pgen_rated

print( "Max instantaneous power per turbine = %.2f MW" % (P_turb_max/1E6) )
print()
print( "Mean turbine power    = %.2f MW" % (P_turb_mean*PP_model.n_turbs/1E6) )
print( "Mean electrical power = %.2f MW" % (P_gen_mean*PP_model.n_turbs/1E6) )
print()
print( "Capacity factor = %.2f" % C_fac )

In [ ]:
def update_TS( change: Tuple[float, float] ) -> None:

    fig, ax = mpl.subplots( 4, 1, figsize = ( 12, 9 ) )
    fig.subplots_adjust( bottom = 0.2, hspace = 0.12 )

    ax[0].plot( hours_vec, z_vec, label = r'Basin: $z$ [m]', dashes=(7,1,1,1) )
    ax[0].plot( hours_vec, ζ_vec, label = r'Tide: $\zeta$ [m]', dashes=(9,1) )
    ax[0].plot( hours_vec, h_vec, label = r'Head: $h$ [m]', dashes=(9,0) )
    ax[0].plot( hours_vec, FSM_states_vec, label = r'State: $S_i$ [-]' )
    ax[0].axhline( 0, color = 'k' )
    ax[0].legend( loc = 'lower left', fontsize = 12 )
    ax[0].grid();

    ax[1].plot( hours_vec, P_avail_vec / 1E6, label = r'$P_\mathrm{avail}$ [MW]'  )
    ax[1].plot( hours_vec, P_turb_vec / 1E6, label = r'$P_\mathrm{turb}$ [MW]'  )
    ax[1].plot( hours_vec, P_gen_vec / 1E6, label = r'$P_\mathrm{gen}$ [MW]', dashes = (9,1) )
    ax[1].legend( loc = 'lower left', fontsize = 12 )
    ax[1].grid()

    ax[2].plot( hours_vec, Q_turb_vec, label = r'$Q_\mathrm{turb}$ [m$^3$/s]' )
    ax[2].plot( hours_vec, Q_sluice_vec, label = r'$Q_\mathrm{sgate}$ [m$^3$/s]', dashes = (9,1) )
    ax[2].legend( loc = 'lower left', fontsize = 12 )
    ax[2].grid()

    ax[3].plot( hours_vec, η_turb_vec, label = r'$\eta_\mathrm{turb}$' )
    ax[3].plot( hours_vec, η_gen_vec, label = r'$\eta_\mathrm{gen}$', dashes = (9,1) )
    ax[3].plot( hours_vec, η_turb_vec*η_gen_vec, label = r'$\eta_\mathrm{turb}\,\eta_\mathrm{gen}$', dashes = (7,1,1,1) )
    ax[3].legend( loc = 'lower left', fontsize = 12 )
    ax[3].grid()

    ax[0].set_xticklabels( [] )
    ax[1].set_xticklabels( [] )
    ax[2].set_xticklabels( [] )
    ax[3].set_yticks( np.arange( 0, 1.01, 0.1 ) )
    ax[3].set_xlabel( 'time [hours]' )

    ax[0].set_xlim( *change )
    ax[1].set_xlim( *change )
    ax[2].set_xlim( *change )
    ax[3].set_xlim( *change )

    mpl.savefig('TimeSeries.pdf', bbox_inches = 'tight', pad_inches = 0 ) # pdflatex
    mpl.savefig('TimeSeries.svg', bbox_inches = 'tight', pad_inches = 0 ) # Word

ipywidgets.interact( update_TS,
                     change = ipywidgets.FloatRangeSlider(
                                value = xdelta, \
                                min = xmin,
                                max = xmax,
                                step = 1,
                                description = "Interval",
                                layout = ipywidgets.Layout( width = '700px' ),
                                readout_format = 'd'
                            )
                    );

In [ ]:
def update_QS( change: Tuple[float, float] ) -> None:

    fig, ax = mpl.subplots( 2, 1, figsize = ( 12, 4.5 ) )
    fig.subplots_adjust( bottom = 0.2, hspace = 0.12 )

    ax[0].plot( hours_vec, n11_vec, color = linecolors[0], label = r'$n_{11}$ [-]' )
    ax[0].legend( loc = 'lower left', fontsize = 12 )
    ax[0].grid();

    ax[1].plot( hours_vec, Q11_vec, color = linecolors[1], label = r'$Q_{11}$ [-]' )
    ax[1].legend( loc = 'lower left', fontsize = 12 )
    ax[1].grid()

    ax[0].set_xticklabels( [] )
    ax[1].set_xlabel( 'time [hours]' )

    ax[0].set_xlim( *change )
    ax[1].set_xlim( *change )

    mpl.savefig('TimeSeries_11.pdf', bbox_inches = 'tight', pad_inches = 0 ) # pdflatex
    mpl.savefig('TimeSeries_11.svg', bbox_inches = 'tight', pad_inches = 0 ) # Word

ipywidgets.interact( update_QS,
                     change = ipywidgets.FloatRangeSlider(
                                value = xdelta, \
                                min = xmin,
                                max = xmax,
                                step = 1,
                                description = "Interval",
                                layout = ipywidgets.Layout( width = '700px' ),
                                readout_format = 'd'
                            )
                    );